In [ ]:
# 3_sentiment_lstm_model.ipynb
# Purpose: Train a BiLSTM model for sentiment classification on Chinese hotel reviews

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.model_selection import train_test_split
import os

In [ ]:

# Load processed data from previous notebook
# Assumes train_pad and train_labels are available from preprocessing
train_pad = np.load("data/train_pad.npy")
train_labels = np.load("data/train_labels.npy")
embedding_matrix = np.load("data/embedding_matrix.npy")


In [ ]:
# Parameters
max_tokens = train_pad.shape[1]       # Sequence length
embedding_dim = embedding_matrix.shape[1]
num_words = embedding_matrix.shape[0] # Vocabulary size

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(train_pad, train_labels, test_size=0.1, random_state=42)


In [ ]:
# Build the BiLSTM model
model = Sequential()
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(LSTM(units=16, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Set up callbacks
log_dir = "logs/sentiment_lstm"
os.makedirs(log_dir, exist_ok=True)

checkpoint_path = os.path.join(log_dir, "sentiment_checkpoint.keras")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1),
    TensorBoard(log_dir=log_dir)
]

In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=5,
                    batch_size=128,
                    callbacks=callbacks)


In [ ]:
# Evaluate on the test set
results = model.evaluate(X_test, y_test)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# Save the final model
model.save(os.path.join(log_dir, "final_sentiment_model.h5"))